# Домашки



## HW01

Просмотрел ноутбуки, в которых было `< 200/200`. Если есть вопросы, пишите в личку.

```python
CustomResNet(
    (model): Sequential(
    (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (0): ResBlock(
        (conv1): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): ResBlock(
        (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(32, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (2): ResBlock(
        (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU()
        (conv2): Conv2d(32, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (3): Flatten(start_dim=1, end_dim=-1)
      (4): Linear(in_features=784, out_features=10, bias=True)
      (5): Softmax(dim=None)
    )
  )
```

Не всегда большие сети --- это хорошо. Если задача простая, не обязательно использовать модели из `torchvision`
```

```


## Высокоуровнкевые фреймворки

https://catalyst-team.github.io/catalyst/

https://github.com/pytorch/ignite

https://github.com/PyTorchLightning/pytorch-lightning



 **Важно**: аккуратнее в сложных задачах, во фреймворках могут быть баги


# Другие задачи CV

* [Ноутбук Захаркина Ильи](https://colab.research.google.com/drive/17iJcJkegjb8fRCpDD9j0M6ua7pLlOzpL?usp=sharing)
* [Запись сема Захаркина](https://www.youtube.com/watch?v=XSPYe4-y4HE)
* [Семантическая сегментация от Татьяны Гайнцевой](https://www.youtube.com/watch?v=tIqndofykgc) (в описании видоса есть ссылки на другие части про сегментацию)


